In [1]:
import zipfile
zFile = zipfile.ZipFile("ml-latest-small.zip", "r")
#ZipFile.namelist(): 获取ZIP文档内所有文件的名称列表
for fileM in zFile.namelist(): 
    zFile.extract(fileM)
"""
ratings.csv 是评分
tags.csv 是电影的标签
"""

'\nratings.csv 是评分\ntags.csv 是电影的标签\n'

In [2]:
import pandas as pd
all_ratings = pd.read_csv("ml-latest-small/ratings.csv")
all_ratings["timestamp"] = pd.to_datetime(all_ratings['timestamp'],unit='s')

In [3]:
from eplot import eplot
df = all_ratings["rating"].value_counts()
df.eplot.bar(title='柱形图')

In [4]:
# 评分大于3分表示喜欢这个电影
all_ratings["like"] = all_ratings["rating"]>=4

In [5]:
train_num = 200
# 训练数据
train_ratings = all_ratings[all_ratings['userId'].isin(range(train_num))]

In [6]:
like_ratings = train_ratings[train_ratings["like"] == True]

In [7]:
# 每一个人喜欢哪一些电影
like_by_user = dict((k,frozenset(v.values)) for k,v in like_ratings.groupby("userId")["movieId"])

In [8]:
# 电影被人喜欢的数量
num_like_of_movie = like_ratings[["movieId","like"]].groupby("movieId").sum()

In [9]:
frequent_itemsets = {}
min_support = 50
# first step 步骤一：生成初始的频繁数据集
frequent_itemsets[1] = dict((frozenset((movie_id,)),row["like"])
                            for movie_id,row in num_like_of_movie.iterrows()
                            if row["like"] > min_support)

In [10]:
# 步骤②③，
from collections import defaultdict
def find_new_frequent_items(movies_like_by_user,frequent_of_k,min_support):
    """
        movies_like_by_user:用户喜欢电影的集合
        frequent_of_k：超集
        min_support:最小的支持度
    """
    counts = defaultdict(int)
    # 获得用户看movie的集合
    for user,movie_ids in movies_like_by_user.items(): 
        # 遍历超集中间的数据项
        for itemset in frequent_of_k:
            # 如数据项在用户的movie集合中，则代表用户同时喜欢这几部电影
            if itemset.issubset(movie_ids):
                # 遍历出现在movie集合但是没有出现在数据项中间的数据
                for other_movie in movie_ids - itemset:
                    current_superset = itemset | frozenset((other_movie,))
                    counts[current_superset] += 1
    return dict([(itemset,support) for itemset,support in counts.items()
                            if support >= min_support])

In [11]:
for k in range(2,10):
    current_set = find_new_frequent_items(like_by_user,frequent_itemsets[k-1],min_support)
    if len(current_set) ==0:
        print("{}项生成的备选项集长度为0，不再进行生成".format(k))
        break
    else:
        print("准备进行{}项生成备选项集".format(k))
        frequent_itemsets[k] = current_set
        
del frequent_itemsets[1]

准备进行2项生成备选项集
准备进行3项生成备选项集
准备进行4项生成备选项集
准备进行5项生成备选项集
准备进行6项生成备选项集
7项生成的备选项集长度为0，不再进行生成


In [12]:
# 生成规则
rules = []
for k,item_counts in frequent_itemsets.items():
    # k代表项数，item_counts代表里面的项
    for item_set in item_counts.keys():
        for item in item_set:
            premise = item_set - set((item,))
            rules.append((premise,item))

In [13]:
# 得到每一条规则在训练集中的应验的次数
# 应验
right_rule = defaultdict(int)
# 没有应验
out_rule = defaultdict(int)

for user,movies in like_by_user.items():
    for rule in rules:
        # premise,item代表购买了premise就会购买item
        premise,item = rule
        if premise.issubset(movies):
            if item in movies:
                right_rule[rule] +=1
            else:
                out_rule[rule] += 1

In [14]:
# 计算每一条规则的置信度
rule_confidence = {rule:right_rule[rule]/float(right_rule[rule] + out_rule[rule]) for rule in rules}
from operator import itemgetter
# 进行从大到小排序
sort_confidence = sorted(rule_confidence.items(),key=itemgetter(1),reverse = True)


In [15]:
# 计算X在训练集中出现的次数
item_num = defaultdict(int)
for user,movies in like_by_user.items():
    for rule in rules:
        premise,item = rule
        if item in movies:
            item_num[rule] += 1
            
# 计算P(X) item_num[rule]代表的就是P(X)
item_num = {k: v/len(like_by_user) for k, v in item_num.items()}   

In [16]:
# 计算每一条规则的Lift
rule_lift = {rule:(right_rule[rule]/(float(right_rule[rule] + out_rule[rule])))/item_num[rule] for rule in rules}
from operator import itemgetter
sort_lift = sorted(rule_lift.items(),key=itemgetter(1),reverse = True)

In [17]:
# 去除训练使用的数据集得到测试集
ratings_test  = all_ratings.drop(train_ratings.index)
# 去除测试集中unlike数据
like_ratings_test = ratings_test[ratings_test["like"]]
user_like_test = dict((k,frozenset(v.values)) for k,v in like_ratings_test.groupby("userId")["movieId"])

In [18]:
for i in range(0,20):
    # 应验
    right_rule = 0
    # 没有应验
    out_rule = 0
    for movies in user_like_test.values():
        if(sort_lift[i][0][0].issubset(movies)):
            if(sort_lift[i][0][1] in movies):
                right_rule +=1
            else:
                out_rule +=1
    print("sort_lift验证的正确度为：{}".format(right_rule/(right_rule+out_rule)))

sort_lift验证的正确度为：0.7872340425531915
sort_lift验证的正确度为：0.75
sort_lift验证的正确度为：0.8837209302325582
sort_lift验证的正确度为：0.7222222222222222
sort_lift验证的正确度为：0.78125
sort_lift验证的正确度为：0.9032258064516129
sort_lift验证的正确度为：0.9142857142857143
sort_lift验证的正确度为：0.7333333333333333
sort_lift验证的正确度为：0.75
sort_lift验证的正确度为：0.7352941176470589
sort_lift验证的正确度为：0.9122807017543859
sort_lift验证的正确度为：0.9259259259259259
sort_lift验证的正确度为：0.7761194029850746
sort_lift验证的正确度为：0.746268656716418
sort_lift验证的正确度为：0.7567567567567568
sort_lift验证的正确度为：0.8125
sort_lift验证的正确度为：0.723404255319149
sort_lift验证的正确度为：0.7241379310344828
sort_lift验证的正确度为：0.7297297297297297
sort_lift验证的正确度为：0.9512195121951219


In [19]:
for j in range(0,20):
    # 应验
    right_rule = 0
    # 没有应验
    out_rule = 0
    for movies in user_like_test.values():
        if(sort_confidence[j][0][0].issubset(movies)):
            if(sort_confidence[j][0][1] in movies):
                right_rule +=1
            else:
                out_rule +=1
    print("sort_confidence验证的正确度为：{}".format(right_rule/(right_rule+out_rule)))

sort_confidence验证的正确度为：0.9047619047619048
sort_confidence验证的正确度为：0.9607843137254902
sort_confidence验证的正确度为：0.875
sort_confidence验证的正确度为：1.0
sort_confidence验证的正确度为：0.9512195121951219
sort_confidence验证的正确度为：0.6764705882352942
sort_confidence验证的正确度为：0.9411764705882353
sort_confidence验证的正确度为：0.9375
sort_confidence验证的正确度为：0.9333333333333333
sort_confidence验证的正确度为：0.9411764705882353
sort_confidence验证的正确度为：0.9259259259259259
sort_confidence验证的正确度为：1.0
sort_confidence验证的正确度为：1.0
sort_confidence验证的正确度为：0.8524590163934426
sort_confidence验证的正确度为：0.896551724137931
sort_confidence验证的正确度为：0.8837209302325582
sort_confidence验证的正确度为：0.8571428571428571
sort_confidence验证的正确度为：0.875
sort_confidence验证的正确度为：0.8222222222222222
sort_confidence验证的正确度为：0.8727272727272727
